In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
data = pd.read_csv('/content/drive/MyDrive/nlp project/data.csv')

In [4]:
data['y'] = data['y'] - 1

In [5]:
data.head()

,Unnamed: 0,targetCategory,post,y
0,0,race,RT @iBeZo: Stupid fucking nigger LeBron. You f...,0
1,1,race,I need a girl from Jamaica I can't fuck with t...,0
2,2,gender,Molly make bitches crazy,1
3,3,race,This niggga said I be branding bitches &#12856...,0
4,4,race,It's funny when u get a black female mad they ...,0


In [6]:
raw_data = {}

In [7]:
raw_data['train'], raw_data['test'] = train_test_split(data, test_size=0.33, random_state=42)

In [8]:
! pip install openprompt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [9]:
from openprompt.data_utils import InputExample

dataset = {}
for split in ['train', 'test']:
    dataset[split] = []
    for i in (raw_data[split]['Unnamed: 0']):
        input_example = InputExample(text_a = raw_data[split]['post'][i], 
                                     label=int(raw_data[split]['y'][i]), 
                                     guid=int(raw_data[split]['Unnamed: 0'][i]))
        dataset[split].append(input_example)
print(dataset['train'][0])

{
  "guid": 9014,
  "label": 2,
  "meta": {},
  "text_a": "abdul , my neighbour was crying . . . abdul , my neighbour , was crying,\"what 's wrong ? \" i asked . \" we lost our baby daughter , cot death , \" he replied . \" oh dear , that 's so sad , never mind abdul , \" i said . \" just think of her as another virgin in paradise for your brave soldiers now . \" _ <elong> xpost : r / sickipedia edit : sorry , i am not good at formatting . suggestions would be helpful for future .",
  "text_b": "",
  "tgt_text": null
}



In [10]:
from openprompt.plms import load_plm
plm, tokenizer, model_config, WrapperClass = load_plm("bert", "bert-base-uncased")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [11]:
from openprompt.prompts import MixedTemplate
promptTemplate = MixedTemplate(
    model=plm,
    text = '{"placeholder":"text_a"}{"soft"} {"soft"} {"soft"} {"mask"}{"soft"}',
    tokenizer = tokenizer,
)

wrapped_example = promptTemplate.wrap_one_example(dataset['train'][0])
print(wrapped_example)

[[{'text': 'abdul , my neighbour was crying . . . abdul , my neighbour , was crying,"what \'s wrong ? " i asked . " we lost our baby daughter , cot death , " he replied . " oh dear , that \'s so sad , never mind abdul , " i said . " just think of her as another virgin in paradise for your brave soldiers now . " _ <elong> xpost : r / sickipedia edit : sorry , i am not good at formatting . suggestions would be helpful for future .', 'soft_token_ids': 0, 'loss_ids': 0, 'shortenable_ids': 1}, {'text': '', 'soft_token_ids': 1, 'loss_ids': 0, 'shortenable_ids': 0}, {'text': '', 'soft_token_ids': 2, 'loss_ids': 0, 'shortenable_ids': 0}, {'text': '', 'soft_token_ids': 3, 'loss_ids': 0, 'shortenable_ids': 0}, {'text': '<mask>', 'soft_token_ids': 0, 'loss_ids': 1, 'shortenable_ids': 0}, {'text': '', 'soft_token_ids': 4, 'loss_ids': 0, 'shortenable_ids': 0}], {'guid': 9014, 'label': 2}]


In [12]:
from openprompt import PromptDataLoader

train_dataloader = PromptDataLoader(dataset=dataset["train"], template=promptTemplate, tokenizer=tokenizer,
    tokenizer_wrapper_class=WrapperClass, max_seq_length=100, decoder_max_length=3,
    batch_size=64,shuffle=True, teacher_forcing=False, predict_eos_token=False,
    truncate_method="head")

tokenizing: 8093it [00:10, 777.89it/s]


In [13]:
from openprompt.prompts import ManualVerbalizer
promptVerbalizer = ManualVerbalizer(
    num_classes=7,
    label_words = [
        ["race","nigger","niglets","niggas"],
        ["gender","women", "hoes", "bitches"],
        ["culture","hillbilly","Muslim"],
        ["victem","refugees","beat", "kill"],
        ["disabled","retarded","retard"],
        ["social","democrats","republicans","ghetto","redneck"],
        ["body","fat"]
    ],
    tokenizer = tokenizer,
)

In [14]:
from openprompt import PromptForClassification
import torch

use_cuda = True
prompt_model = PromptForClassification(plm=plm,template=promptTemplate, verbalizer=promptVerbalizer, freeze_plm=False)
if use_cuda:
    prompt_model=  prompt_model.cuda()

In [15]:
# ## below is standard training
from transformers import  AdamW, get_linear_schedule_with_warmup
loss_func = torch.nn.CrossEntropyLoss()

no_decay = ['bias', 'LayerNorm.weight']

# it's always good practice to set no decay to biase and LayerNorm parameters
optimizer_grouped_parameters1 = [
    {'params': [p for n, p in prompt_model.plm.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in prompt_model.plm.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

# Using different optimizer for prompt parameters and model parameters
optimizer_grouped_parameters2 = [
    {'params': [p for n,p in prompt_model.template.named_parameters() if "raw_embedding" not in n]}
]

optimizer1 = AdamW(optimizer_grouped_parameters1, lr=1e-4)
optimizer2 = AdamW(optimizer_grouped_parameters2, lr=1e-3)

for epoch in range(10):
    tot_loss = 0
    for step, inputs in enumerate(train_dataloader):
        if use_cuda:
            inputs = inputs.cuda()
        logits = prompt_model(inputs)
        labels = inputs['label']
        loss = loss_func(logits, labels)
        loss.backward()
        tot_loss += loss.item()
        optimizer1.step()
        optimizer1.zero_grad()
        optimizer2.step()
        optimizer2.zero_grad()
        print(tot_loss/(step+1))

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


1.841371774673462
4.282837986946106
3.5545215606689453
3.1090999245643616
2.8648671865463258
2.681787371635437
2.551536883626665
2.434072971343994
2.3509222401512995
2.2817702293395996
2.2297827222130517
2.159864693880081
2.09205032311953
2.0387223533221652
2.0047117710113525
1.9622924104332924
1.9115468614241655
1.8755185736550226
1.833646479405855
1.794433557987213
1.7559493439538139
1.7280640981414102
1.6862946956054023
1.642171325782935
1.6133701300621033
1.5910658079844255
1.5727916448204606
1.552625100527491
1.530291107194177
1.4996239125728608
1.4837900534752877
1.4681407622992992
1.4495922110297463
1.4273402094841003
1.4075874498912266
1.395878889494472
1.3798364948582005
1.3665068714242232
1.3473327924043705
1.3333429500460625
1.3154715037927396
1.3047788370223272
1.2919729060904925
1.2770996703342958
1.2656434469752842
1.2586325938286989
1.2467915846946391
1.2336370709041755
1.2223348228298887
1.2135663437843323
1.2023335379712723
1.1933608570924172
1.1833573759726759
1.17160

In [16]:
# ## evaluate

# %%
validation_dataloader = PromptDataLoader(dataset=dataset["test"], template=promptTemplate, tokenizer=tokenizer,
    tokenizer_wrapper_class=WrapperClass, max_seq_length=100, decoder_max_length=3,
    batch_size=4,shuffle=False, teacher_forcing=False, predict_eos_token=False,
    truncate_method="head")


allpreds = []
alllabels = []
for step, inputs in enumerate(validation_dataloader):
    if use_cuda:
        inputs = inputs.cuda()
    logits = prompt_model(inputs)
    labels = inputs['label']
    alllabels.extend(labels.cpu().tolist())
    allpreds.extend(torch.argmax(logits, dim=-1).cpu().tolist())

acc = sum([int(i==j) for i,j in zip(allpreds, alllabels)])/len(allpreds)
print(acc)

tokenizing: 3987it [00:03, 1064.29it/s]


0.7587158264359167
